In [1]:
import pandas as pd

In [142]:
tweet_df = pd.read_csv("./data/tweets2.csv")
print(len(tweet_df))

tweet_df = tweet_df[tweet_df.text.map(lambda x: x.isascii())]
print(len(tweet_df))
tweet_df = tweet_df.drop_duplicates()
print(len(tweet_df))


76037
38251
37282


In [113]:
def remove_special_char(row):
    text = row['text']
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("\r", " ")
    return text

In [114]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def getLabel(row):
    text = row['text']
    label = ""
    words = text.split(" ")
    for w in words:
        if '#' in w and len(w) > 1:
            # print(w)
            label = w[w.index('#')+1:]
            for ele in label:
                    if ele in punc:
                        label = label.replace(ele, "")
            break
    return label

In [162]:
def get_new_text(row):
    text = row['text']
    label = row['label']
    new_text = []
    words = text.split(" ")
    for w in words:
        if w in stopword:
            continue
        # w = label if ("#" + label) in w else w
        w = '@user' if w.startswith('@') and len(w) > 1 else w
        w = 'http' if 'http' in w else w
        new_text.append(w)
    return " ".join(new_text)

In [125]:
import nltk 
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')


def get_mask(row):
    text = row['text']
    new_text = []
    words = text.split(" ")
    for w in words:
        if w in stopword:
            continue
        # print(w)
        w = '[MASK]' if '#' in w else w
        w = '@user' if w.startswith('@') and len(w) > 1 else w
        w = 'http' if 'http' in w else w
        new_text.append(w)
    return " ".join(new_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vivian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [163]:
tweet_df['text'] = tweet_df.apply(remove_special_char, axis=1)
# print(tweet_df)
tweet_df['label'] = tweet_df.apply(getLabel, axis=1)
tweet_df['cleaned_tweet'] = tweet_df.apply(get_new_text, axis=1)
tweet_df['masked_tweet'] = tweet_df.apply(get_mask, axis=1)
tweet_df = tweet_df.drop_duplicates()
print(len(tweet_df))
tweet_df = tweet_df.drop_duplicates(['masked_tweet'])
print(len(tweet_df))
print(tweet_df['masked_tweet'])

31055
31055
1        Is mora?  Being cute kill perfect Meow Meow Hu...
52       Spy x family Ver. Aether&amp;Ayaka  [MASK] [MA...
56       Ayaka!! But swimsuit style. &lt;33 (I'm still ...
61               Simplemente, Kamisato Ayaka.  [MASK] http
66       arriesge banner de ayato solo para acercame al...
                               ...                        
76017    Penales: El gol de Caruso la atajada de Wysock...
76023    [MASK] [MASK] [MASK]   GOOOOOOOOLLLLLLLL de [M...
76027    @user Great show always dudes!! Definitely add...
76028      @user Buona Pasqua dai mercatini di [MASK] http
76032    [MASK] Giuseppe Caruso - La Vita Nel Sud feat....
Name: masked_tweet, Length: 31055, dtype: object


In [161]:
result_df = tweet_df[['cleaned_tweet', 'label']]
result_df.to_csv("./data/labeled_tweet2.csv", index=False)

masked_df = tweet_df[['masked_tweet']]
masked_df.to_csv("./data/masked_tweet2.csv", index=False)
